In [1]:
import pandas as pd
import numpy as np
import requests as re
import geopandas as gpd
import json
import warnings
import requests
import time
import folium
import sys


from shapely.prepared import prep
from shapely.geometry import Point,Polygon
from shapely.ops import transform
from scipy.spatial import Delaunay
from keplergl import KeplerGl

sys.path.insert(1, '../config/')
import kepler_configs


warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

In [2]:
api_key = json.load(open('../config/api_keys.json',))['key_hisham']

In [11]:
def func_transform_api_result(row):
    geometry = row.geometry['location']
    lat,lon = geometry['lat'],geometry['lng']
    types = row.types
    first_type = types[0]
    second_type = types[1]
    return lat,lon,first_type,second_type

def func_get_places_poi(lat,lon,resolution):
    
    url = """https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lon}
    &radius={resolution}
    &type=point_of_interest
    &key={api_key}""".format(api_key=api_key,lat=lat,lon=lon,resolution=resolution)

    payload={}
    headers = {}
    params = {}

    response = requests.request("GET", url, headers=headers, data=payload)
    places_df = []

    results = json.loads(response.content)

    places_df.append(results['results'])

    while 'next_page_token' in results:
        time.sleep(1)
        params['pagetoken'] = results['next_page_token']
        response = requests.get(url,params=params)
        results = json.loads(response.content)
        places_df.append(results['results'])
        
    final_df = pd.concat([pd.DataFrame(df) for df in places_df])
    final_df = final_df.reset_index(drop=True)
    
    return final_df


def func_flip(x, y):
    """Flips the x and y coordinate values"""
    return y, x

In [4]:
# lahore_vor = gpd.read_file('../data/vector/dev/LAHORE/lahore_voronoi_fishnet_1000m.shp')
# lahore_vor_stats = pd.read_csv('../data/summary-stats/lhr_pop_vor.csv')

# lahore_vor = lahore_vor.merge(lahore_vor_stats)
# lahore_vor = lahore_vor.sort_values(by='sum_pop',ascending=False).reset_index(drop=True)
# lahore_vor['centroid'] = lahore_vor.geometry.centroid

In [12]:
lahore_vor = gpd.read_file('../data/vector/dev/LAHORE/gmaps_places/lahore_voronoi_fishnet_1000m_updated_fix.shp')
lahore_vor['centroid'] = lahore_vor.geometry.centroid

In [13]:
# for ind,row in lahore_vor.iterrows():
#     if(ind<=159):
#         lahore_vor.loc[ind,'fetched']=1
#     else:
#         lahore_vor.loc[ind,'fetched']=0

# for ind,row in lahore_vor.iterrows():
#     if(ind>=160 and ind<349):
#         lahore_vor.loc[ind,'fetched']=0

In [15]:
len(lahore_vor[lahore_vor.fetched==0])

62

#### get lahore overall boundary

In [14]:
lahore_dist_boundary = gpd.read_file('../data/vector/pak_administrative_shapefiles/District_Boundary.shp')
lahore_dist_boundary = lahore_dist_boundary[lahore_dist_boundary.DISTRICT=='LAHORE']
lahore_dist_boundary['geometry'] = lahore_dist_boundary.geometry.apply(lambda x: transform(func_flip,x))
lahore_dist_boundary = lahore_dist_boundary[lahore_dist_boundary.DISTRICT=='LAHORE'].geometry.values[0]

### ALGO

In [16]:
#uses Delaunay to break down a polygon via triangulation, sends api request for centroid of each new polygon until
#a point of saturation is reached i.e further breaking down is not givin more than 60 api results.
#currently hardcorded to go only 4 levels deep
rdf_list = []
centroids = []
requests_counter = 0
limit = 2500
for ind, row in lahore_vor.iterrows():
    if(row.fetched!=1):
        left = len(lahore_vor)-ind
        out_str = 'requests: {req} & vor_left: {left}'.format(req=requests_counter,left=left)
        sys.stdout.write(out_str)
        sys.stdout.flush()
        resolution = 1500
        level = 1

        #LEVEL=1
        coords = row.centroid.xy
        lat,lon = coords[0][0],coords[1][0]
        centroids.append(np.array([lat,lon]))
        #print(np.array([lat,lon]))
        requests_counter+=3
        if(requests_counter<=limit):
            rdf = func_get_places_poi(lat=lat,lon=lon,resolution=resolution)
        else:
            print('LIMIT REACHED!')
            sys.exit()
            break
        rdf_list.append(rdf)

        #LEVEL=2
        if(len(rdf)==60):
            #print('reached L2')
            poly = row.geometry
            poly_points_list = []
            if(poly.geom_type=='MultiPolygon'):
                poly_points = np.array(poly[0].exterior.coords)
                poly_points_list.append(poly_points)
#                 poly_points = [np.array(pol.exterior.coords) for pol in poly]
#                 poly_points = np.concatenate(poly_points,axis=0)
#                 poly_points_list.append(poly_points)
            elif(poly.geom_type=='Polygon'):   
                poly_points = np.array(poly.exterior.coords)
                poly_points_list.append(poly_points)
            tri = Delaunay(poly_points_list[0])
            polys = poly_points[tri.simplices]
            polys = [Polygon(poly) for poly in polys]
            if(len(polys)>1):
                level += 1
                for poly in polys:
                    centroid = poly.centroid
                    #checks if new centroid in Lahore
                    if(lahore_dist_boundary.contains(centroid) == True):
                        centroid = np.array(poly.centroid.coords)
                        #print(centroid[0])
                        centroids.append(centroid[0])
                        lat,lon= centroid[0][0],centroid[0][1]
                        requests_counter+=3
                        if(requests_counter<=limit):
                            rdf = func_get_places_poi(lat=lat,lon=lon,resolution=resolution/level)
                        else:
                            print('LIMIT REACHED!')
                            sys.exit()
                            break
                        rdf_list.append(rdf)
                    else:
                        pass

                    #LEVEL=3 
                    if(len(rdf)==60):
                        #print('reached L3')
                        poly_points = np.array(poly.exterior.coords)
                        poly_points = np.concatenate((poly_points,centroid),axis=0)
                        tri = Delaunay(poly_points)
                        l2_polys = poly_points[tri.simplices]
                        l2_polys = [Polygon(poly) for poly in l2_polys]
                        if(len(l2_polys)>1):
                            for poly in l2_polys:
                                level += 1
                                centroid = poly.centroid
                                if(lahore_dist_boundary.contains(centroid)==True):
                                    centroid = np.array(centroid.coords)
                                    #print(centroid[0])
                                    centroids.append(centroid[0])
                                    lat,lon= centroid[0][0],centroid[0][1]
                                    requests_counter+=3
                                    if(requests_counter<=limit):
                                        rdf = func_get_places_poi(lat=lat,lon=lon,resolution=resolution/level)
                                    else:
                                        print('LIMIT REACHEAD!')
                                        sys.exit()
                                        break
                                    rdf_list.append(rdf)
                                else:
                                    pass

                                #LEVEL=4
                                if(len(rdf)==60):
                                    #print('reached L4')
                                    poly_points = np.array(poly.exterior.coords)
                                    poly_points = np.concatenate((poly_points,centroid),axis=0)
                                    tri = Delaunay(poly_points)
                                    l3_polys = poly_points[tri.simplices]
                                    l3_polys = [Polygon(poly) for poly in l2_polys]
                                    for poly in l3_polys:
                                        level += 1
                                        centroid = poly.centroid
                                        if(lahore_dist_boundary.contains(centroid)==True):
                                            centroid = np.array(centroid.coords)
                                            #print(centroid[0])
                                            centroids.append(centroid[0])
                                            lat,lon= centroid[0][0],centroid[0][1]
                                            requests_counter+=3
                                            if(requests_counter<=limit):
                                                rdf = func_get_places_poi(lat=lat,lon=lon,resolution=resolution/level)
                                            else:
                                                print('LIMIT REACHED!')
                                                sys.exit()
                                                break

                                            rdf_list.append(rdf)
                                        else:
                                            pass
        temp_dfs = pd.concat(rdf_list)
        fname = '../data/dev/vor_latest2.pkl'
        temp_dfs.to_pickle(fname)
        
        lahore_vor.loc[ind,'fetched']=1
        temp = lahore_vor
        temp = temp.drop(columns=['centroid'])
        temp.to_file('../data/vector/dev/LAHORE/gmaps_places/lahore_voronoi_fishnet_1000m_updated_fix2.shp')
        
    elif(row.fetched==1):
        print('fetched')
            


fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched
fetched


In [17]:
#check how many poi
rdfs = pd.concat(rdf_list)
len(rdfs.place_id.unique())

1208

### POST ALGO CLEANUP, SAVE, & VISUALIZATION

In [21]:
#poi
#rdfs = pd.read_pickle('../data/vector/dev/LAHORE/gmaps_places/10vor_test/10vor_poi.pkl')
rdfs = rdfs.drop_duplicates(subset=['place_id'])
rdfs[['lat','lon','first_type','second_type']] = rdfs.apply(func_transform_api_result,
                                                                    axis=1,
                                                                    result_type='expand')
lahore_poi = rdfs

lahore_poi_gpd = lahore_poi[['place_id','name','user_ratings_total','rating',
                                       'price_level','first_type','second_type','lat','lon']]
def func_temp(row):
    return(Point(row.lat,row.lon))
lahore_poi_gpd['geometry'] = lahore_poi_gpd.apply(func_temp,axis=1)
lahore_poi_gpd = gpd.GeoDataFrame(lahore_poi_gpd,geometry='geometry')
lahore_poi_gpd.to_file('../data/vector/dev/LAHORE/gmaps_places/lahore_ind4.shp')
#lahore_poi_gpd.to_file('../data/vector/dev/LAHORE/gmaps_places/lahore_ind3.shp')
#lahore_poi_gpd.to_file('../data/vector/dev/LAHORE/gmaps_places/lahore_ind160.shp')

#we have saved up till index 160 of lahore tesselations file - these are also flagged as fetched under the 
#lahore tesselations shape files

In [22]:
#vor tesselations
#lahore_vor = lahore_vor.drop(columns='centroid')
lahore_vor = gpd.GeoDataFrame(lahore_vor,geometry='geometry')
#lahore_vor.to_file('../data/vector/dev/LAHORE/gmaps_places/lahore_voronoi_fishnet_1000m_updated.shp')

In [23]:
#centroids 
lahore_centroirds = pd.Series(centroids)
centroids_df = pd.DataFrame(lahore_centroirds)
centroids_df = centroids_df.rename(columns={0:'lat_lon'})
centroids_df['geometry'] = centroids_df.lat_lon.apply(lambda x: Point(x[0],x[1]))
centroids_gpd = gpd.GeoDataFrame(centroids_df,geometry='geometry')
centroids_gpd = gpd.sjoin(centroids_gpd,lahore_vor,how='left')
centroids_gpd = centroids_gpd.drop(columns=['lat_lon','index_right'])
#centroids_gpd.to_csv('../data/vector/dev/LAHORE/gmaps_places/centroids_ind160.csv',index=False)

In [33]:
#flipping coordinates for kepler-gl
lahore_poi_gpd['geometry'] = lahore_poi_gpd.geometry.apply(lambda x: transform(func_flip,x))
centroids_gpd['geometry'] = centroids_gpd.geometry.apply(lambda x: transform(func_flip,x))
lahore_vor['geometry'] = lahore_vor.geometry.apply(lambda x: transform(func_flip,x))

In [25]:
map_ = KeplerGl(height=1000)
map_.add_data(data=lahore_poi_gpd,name='poi')
map_.add_data(data=centroids_gpd,name='centroids')
map_.add_data(data=lahore_vor,name='vors')

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


ValueError: Can't clean for JSON: <shapely.geometry.point.Point object at 0x7f57b09695c0>

In [50]:
map_.save_to_html(file_name='../viz/cells_poi2_places.html',config=kepler_configs.lahore_vor_poi_test)

Map saved to ../viz/cells_poi2_places.html!


### combine lahore_poi

In [27]:
lahore_poi = gpd.read_file('../data/vector/dev/LAHORE/gmaps_places/lahore_ind2.shp')
lahore_poi_2 = gpd.read_file('../data/vector/dev/LAHORE/gmaps_places/lahore_ind160.shp')
lahore_poi_3 = gpd.read_file('../data/vector/dev/LAHORE/gmaps_places/lahore_ind3.shp')
lahore_poi_4 = gpd.read_file('../data/vector/dev/LAHORE/gmaps_places/lahore_ind4.shp')

#lahore_poi_gpd.to_file('../data/vector/dev/LAHORE/gmaps_places/lahore_ind2.shp')


In [28]:
lahore_combined_gpd = pd.concat([lahore_poi,lahore_poi_2,lahore_poi_3,lahore_poi_4]).reset_index(drop=True)

In [42]:
lahore_combined_gpd.drop_duplicates(subset='place_id')

,place_id,name,user_ratin,rating,price_leve,first_type,second_typ,lat,lon,geometry
0,ChIJ2QeB5YMEGTkRYiR-zGy-OsI,Lahore,NaN,NaN,NaN,locality,political,31.520370,74.358747,POINT (31.52037 74.35875)
1,ChIJXUl_8rwSGTkRqaEZK6UVUzI,Wagah,7111.0,4.5,NaN,tourist_attraction,point_of_interest,31.604757,74.574136,POINT (31.60476 74.57414)
2,ChIJm769CLETGTkRmm3x2sHt3iQ,Darbar Shahjamal,3.0,3.3,NaN,mosque,place_of_worship,31.603353,74.563741,POINT (31.60335 74.56374)
3,ChIJLfgtYeITGTkRTf1lrx9kn_g,Azadi Museum,5.0,2.6,NaN,museum,point_of_interest,31.603989,74.564205,POINT (31.60399 74.56421)
4,ChIJcYS4dpcSGTkR6t9fdM9FI_U,National Logistics Cell Office,8.0,3.8,NaN,point_of_interest,establishment,31.602750,74.566538,POINT (31.60275 74.56654)
...,...,...,...,...,...,...,...,...,...,...
60287,ChIJx-mH3xqtGTkRWB8QHOVuJ1M,Karma wala karyana store ?,NaN,NaN,None,food,point_of_interest,31.269862,74.247963,POINT (31.26986 74.24796)
60348,ChIJm210j59XGDkRqDCH15miDpI,Manga,NaN,NaN,None,sublocality_level_2,sublocality,31.306175,74.050575,POINT (31.30617 74.05058)
60423,ChIJbRjlzgipGTkRfJHvpFF23-I,Apartments E block,NaN,NaN,None,point_of_interest,establishment,31.311563,74.367184,POINT (31.31156 74.36718)
60456,ChIJATndfnmrGTkR5i1MgREmOso,ButterScotchINC,NaN,NaN,None,clothing_store,point_of_interest,31.362168,74.275921,POINT (31.36217 74.27592)


In [35]:
sjoined = gpd.sjoin(lahore_vor,lahore_combined_gpd,how='left',op='contains')

In [36]:
sjoined = sjoined.reset_index(drop=True)

In [43]:
sjoined = sjoined.drop_duplicates(subset=['place_id'])

In [46]:
sjoined = sjoined.drop(columns='centroid')

In [47]:
sjoined.to_file('../data/vector/dev/LAHORE/gmaps_places/lahore_all_poi_mapped.shp')

In [50]:
sjoined.columns

Index(['vor_id', 'area_sq', 'area_sqrt', 'sum_pop', 'mean_pop', 'max_pop',
       'min_pop', 'fetched', 'geometry', 'index_right', 'place_id', 'name',
       'user_ratin', 'rating', 'price_leve', 'first_type', 'second_typ', 'lat',
       'lon'],
      dtype='object')

In [57]:
sjoined[['vor_id', 'place_id', 'name',
       'user_ratin', 'rating', 'price_leve', 'first_type', 'second_typ', 'lat',
       'lon']].to_csv('../data/summary-stats/lahore_poi_vor.csv',index=False)